In [15]:
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from gensim.models import LdaModel, LsiModel
from gensim.corpora import Dictionary
from gensim import corpora
import pyLDAvis.gensim
import numpy as np

import matplotlib.pyplot  as plt

In [16]:
file = pd.read_csv('lematizador.txt', sep = '\t', header = None)
file.columns = ['surfice', 'lemma']
file.set_index('surfice', inplace = True)
diccionario = file.to_dict()
dicc = diccionario['lemma']

In [17]:
stopwords = stopwords.words('spanish')+list(string.punctuation)

In [18]:
def lematizar_dic(lista, dicc):
    valores = []
    for x in lista:
        if x in dicc.keys() and x != 'estado':
            valores.append(dicc[x])
        else:
            valores.append(x)
    return(' '.join(valores))   

In [19]:
df = pd.read_csv('CORR_expositores_final_df.csv', sep = ';')

In [20]:
textos = [word_tokenize(x.lower().replace('``','').replace("\'\'",'')) for x in df.texto]

In [21]:
lista2 = []
for x in textos:
    y = lematizar_dic(x, dicc)
    lista2.append(y)

In [22]:
lematizado_plenaria_df = pd.Series(lista2)


In [23]:
nueva_df = pd.concat([df,lematizado_plenaria_df.rename('texto_lematizado')], axis = 1).dropna(subset=['posicion'])#.groupby('posicion')

In [24]:
def procesar_texto(text, stopwords=[]):
    text = word_tokenize(text.lower())
    return [tok for tok in text if tok not in stopwords]

In [25]:
textoslematizados = [procesar_texto(x,stopwords) for x in nueva_df.texto_lematizado]

In [35]:
dictionarylem = corpora.Dictionary(textoslematizados)
print(len(dictionarylem))
dictionarylem.filter_extremes()
print(len(dictionarylem))
corpuslematizado = [dictionarylem.doc2bow(text) for text in textoslematizados]
ldamodel = LdaModel(corpus=corpuslematizado, num_topics=10, id2word=dictionarylem, iterations=4000, passes=10)
ldamodel.print_topics(10, 10)

16232
4544


[(0,
  '0.015*"materno" + 0.015*"salud" + 0.014*"pais" + 0.012*"ciento" + 0.012*"mortalidad" + 0.011*"aborto" + 0.011*"muerte" + 0.007*"embarazar" + 0.006*"argentino" + 0.006*"partir"'),
 (1,
  '0.008*"personar" + 0.007*"deber" + 0.007*"creer" + 0.005*"años" + 0.005*"sociedad" + 0.005*"ir" + 0.005*"sobrar" + 0.005*"politico" + 0.005*"tratar" + 0.005*"existir"'),
 (2,
  '0.013*"embarazar" + 0.011*"hijo" + 0.009*"ir" + 0.007*"años" + 0.006*"salud" + 0.006*"madre" + 0.006*"situacion" + 0.005*"dar" + 0.005*"deber" + 0.005*"dia"'),
 (3,
  '0.015*"articular" + 0.015*"humano" + 0.013*"convencion" + 0.012*"niño" + 0.011*"personar" + 0.010*"constitucion" + 0.010*"constitucional" + 0.010*"deber" + 0.010*"sobrar" + 0.009*"casar"'),
 (4,
  '0.012*"ir" + 0.012*"hablar" + 0.008*"años" + 0.008*"vez" + 0.007*"personar" + 0.007*"semana" + 0.007*"beber" + 0.006*"diputar" + 0.006*"embarazar" + 0.005*"preguntar"'),
 (5,
  '0.019*"salud" + 0.010*"legal" + 0.010*"sexual" + 0.009*"sobrar" + 0.008*"seguro" + 

In [42]:
textos = [procesar_texto(x,stopwords) for x in nueva_df.texto]
textosfavor = [procesar_texto(x,stopwords) for x in nueva_df.texto_lematizado[nueva_df.posicion=='a favor']]
textoscontra = [procesar_texto(x,stopwords) for x in nueva_df.texto_lematizado[nueva_df.posicion=='en contra']]

In [43]:
"""dictionary = corpora.Dictionary(textos)
dictionary.filter_extremes()
corpus = [dictionary.doc2bow(text) for text in textos]
ldamodelcompleto = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, iterations=4000, passes=10)
ldamodelcompleto.print_topics(10, 10)"""

'dictionary = corpora.Dictionary(textos)\ndictionary.filter_extremes()\ncorpus = [dictionary.doc2bow(text) for text in textos]\nldamodelcompleto = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, iterations=4000, passes=10)\nldamodelcompleto.print_topics(10, 10)'

In [44]:
dictionaryfavor = corpora.Dictionary(textosfavor)
print(len(dictionaryfavor))
dictionaryfavor.filter_extremes()
print(len(dictionaryfavor))
corpusfavor = [dictionaryfavor.doc2bow(text) for text in textosfavor]
ldamodelfavor = LdaModel(corpus=corpusfavor, num_topics=10, id2word=dictionaryfavor, iterations=4000, passes=10)
ldamodelfavor.print_topics(10, 10)

10861
2908


[(0,
  '0.016*"hijo" + 0.007*"desear" + 0.007*"pais" + 0.007*"madre" + 0.006*"hablar" + 0.005*"personar" + 0.005*"años" + 0.005*"debatir" + 0.005*"entrar" + 0.005*"decidir"'),
 (1,
  '0.009*"legal" + 0.007*"aborto" + 0.007*"seguro" + 0.007*"interrupcion" + 0.007*"acceso" + 0.006*"pais" + 0.006*"ciento" + 0.006*"años" + 0.006*"practico" + 0.006*"profesional"'),
 (2,
  '0.009*"argentino" + 0.008*"ley" + 0.008*"pais" + 0.008*"deber" + 0.007*"debatir" + 0.007*"politico" + 0.007*"aborto" + 0.006*"hoy" + 0.006*"practico" + 0.006*"cortar"'),
 (3,
  '0.010*"ley" + 0.008*"deber" + 0.006*"años" + 0.006*"hablar" + 0.006*"politico" + 0.006*"debatir" + 0.005*"sexual" + 0.005*"personar" + 0.005*"aborto" + 0.005*"hoy"'),
 (4,
  '0.011*"deber" + 0.009*"constitucional" + 0.008*"constitucion" + 0.008*"acordar" + 0.008*"casar" + 0.007*"hablar" + 0.007*"cortar" + 0.007*"preguntar" + 0.006*"argumentar" + 0.006*"tercero"'),
 (5,
  '0.012*"cuerpo" + 0.010*"``" + 0.008*"deber" + 0.008*"decidir" + 0.007*"decis

In [45]:
dictionarycontra = corpora.Dictionary(textoscontra)
print(len(dictionarycontra))
dictionarycontra.filter_extrems()
print(len(dictionarycontra))
corpuscontra = [dictionarycontra.doc2bow(text) for text in textoscontra]
ldamodelcontra = LdaModel(corpus=corpuscontra, num_topics=10, id2word=dictionarycontra, iterations=4000, passes=10)
ldamodelcontra.print_topics(10, 10)

11724


AttributeError: 'Dictionary' object has no attribute 'filter_extrems'

In [ ]:
import pyLDAvis.gensim
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionarylem, mds="tsne")

In [ ]:
lsimodel = LsiModel(corpus=corpuscontra, num_topics=10, id2word=dictionary)